In [1]:
import requests
from itertools import chain
from collections import namedtuple
import logging

from lxml import etree
from pprint import pprint
from IPython.core.display import display, HTML

from io import StringIO

In [3]:
url = f"https://api.github.com/repos/BSData/wh40k/contents/"
files = requests.get(url).json()
catalogues = []
# 
for file in files:
    if file['type'] == 'file' and file['name'][0] != '.':
        catalogues.append(file['name'])
catalogues

['Aeldari - Craftworlds.cat',
 'Aeldari - Drukhari.cat',
 'Aeldari - FW Corsairs.cat',
 'Aeldari - Harlequins.cat',
 'Aeldari - Ynnari.cat',
 'Chaos - Chaos Knights.cat',
 'Chaos - Chaos Space Marines.cat',
 'Chaos - Daemons.cat',
 'Chaos - Dark Mechanicum.cat',
 'Chaos - Death Guard.cat',
 'Chaos - FW Heretic Astartes.cat',
 'Chaos - FW Renegade and Heretics.cat',
 'Chaos - Gellerpox Infected.cat',
 'Chaos - Servants of the Abyss.cat',
 'Chaos - Thousand Sons.cat',
 'Chaos - Titanicus Traitoris.cat',
 'Fallen.cat',
 'Imperium - Adepta Sororitas.cat',
 'Imperium - Adeptus Astra Telepathica.cat',
 'Imperium - Adeptus Custodes.cat',
 'Imperium - Adeptus Mechanicus.cat',
 'Imperium - Astra Militarum - Library.cat',
 'Imperium - Astra Militarum.cat',
 'Imperium - Black Templars.cat',
 'Imperium - Blackstone Fortress.cat',
 'Imperium - Blood Angels.cat',
 'Imperium - Dark Angels.cat',
 'Imperium - Deathwatch.cat',
 'Imperium - Elucidian Starstriders.cat',
 'Imperium - FW Adeptus Astartes.ca

In [4]:
with open('src/battlescribe/game_schema.xsd', 'rb') as fh:
    game_schema = etree.XMLSchema(etree.fromstring(fh.read()))
with open('src/battlescribe/catelogue_schema.xsd', 'rb') as fh:
    catelogue_schema = etree.XMLSchema(etree.fromstring(fh.read()))

In [5]:
path = 'https://raw.githubusercontent.com/BSData/wh40k/master/{}'.format(
    catalogues[0])
r = requests.get(path)
cat = etree.fromstring(
    r.content, 
    etree.XMLParser(
        remove_blank_text=True, schema = catelogue_schema
    )
)

In [6]:
path = 'https://raw.githubusercontent.com/BSData/wh40k/master/{}'.format('Warhammer 40,000.gst')
r = requests.get(path)
base = etree.fromstring(
    r.content, 
    etree.XMLParser(
        remove_blank_text=True, schema = game_schema
    )
)

In [7]:
for x in cat:
    base.append(x)

{http://www.battlescribe.net/schema/catalogueSchema}publications 1
{http://www.battlescribe.net/schema/catalogueSchema}profileTypes 1
{http://www.battlescribe.net/schema/catalogueSchema}categoryEntries 82
{http://www.battlescribe.net/schema/catalogueSchema}entryLinks 77
{http://www.battlescribe.net/schema/catalogueSchema}sharedSelectionEntries 217
{http://www.battlescribe.net/schema/catalogueSchema}sharedSelectionEntryGroups 13
{http://www.battlescribe.net/schema/catalogueSchema}sharedRules 3
{http://www.battlescribe.net/schema/catalogueSchema}sharedProfiles 116
{http://www.battlescribe.net/schema/catalogueSchema}catalogueLinks 1


In [10]:
for x in base:
    print(x.tag, len(x))

{http://www.battlescribe.net/schema/gameSystemSchema}readme 0
{http://www.battlescribe.net/schema/gameSystemSchema}publications 28
{http://www.battlescribe.net/schema/gameSystemSchema}costTypes 3
{http://www.battlescribe.net/schema/gameSystemSchema}profileTypes 10
{http://www.battlescribe.net/schema/gameSystemSchema}categoryEntries 68
{http://www.battlescribe.net/schema/gameSystemSchema}forceEntries 13
{http://www.battlescribe.net/schema/gameSystemSchema}entryLinks 20
{http://www.battlescribe.net/schema/gameSystemSchema}sharedSelectionEntries 181
{http://www.battlescribe.net/schema/gameSystemSchema}sharedSelectionEntryGroups 3
{http://www.battlescribe.net/schema/gameSystemSchema}sharedRules 17
{http://www.battlescribe.net/schema/gameSystemSchema}sharedProfiles 190
{http://www.battlescribe.net/schema/catalogueSchema}publications 1
{http://www.battlescribe.net/schema/catalogueSchema}profileTypes 1
{http://www.battlescribe.net/schema/catalogueSchema}categoryEntries 82
{http://www.battlesc

In [23]:
for y in base.xpath(
    "//gc:categoryLink[@primary='true']", 
    namespaces={'gc': 'http://www.battlescribe.net/schema/gameSystemSchema'})[:1]:
    print(y.tag, y.attrib)
    for x in y:
        print(x.tag, x.attrib)
        print(etree.tostring(x))
        print()

{http://www.battlescribe.net/schema/gameSystemSchema}categoryLink {'id': '46df-b154-20ff-cce1-d713cda3-5d0f-40d8-b621-69233263ec2a', 'hidden': 'false', 'targetId': 'd713cda3-5d0f-40d8-b621-69233263ec2a', 'primary': 'true'}


In [8]:
# remove duplicates within selection
# group profiles within an entry

In [45]:
with open("src/battlescribe/catelogue.xslt", "r") as f:
    xslt_root = etree.XML(f.read())
transform = etree.XSLT(xslt_root)
result_tree = transform(base)
with open("src/battlescribe/test.html", "w") as f:
    f.write(str(result_tree))
HTML(str(result_tree))